# GEE assets migration

Notebook to migrate GEE assets from one a legacy project to a new cloud project.

## Step 1: Setup
**Authenticate with Earth Engine:**

In [3]:
import ee
ee.Authenticate()
ee.Initialize()

## Step 2: List Legacy Assets

In [32]:

def list_assets(path):
    asset_list = ee.data.listAssets({'parent': path})
    return [asset['name'] for asset in asset_list['assets']]

legacy_project = 'projects/earthengine-legacy/assets/projects/soils-revealed'
legacy_assets = list_assets(legacy_project)

# Print and save the asset list
print(legacy_assets)
with open('legacy_assets.txt', 'w') as file:
    file.write('\n'.join(legacy_assets))

['projects/earthengine-legacy/assets/projects/soils-revealed/ESA_landcover_ipcc', 'projects/earthengine-legacy/assets/projects/soils-revealed/Future', 'projects/earthengine-legacy/assets/projects/soils-revealed/Historic', 'projects/earthengine-legacy/assets/projects/soils-revealed/Recent', 'projects/earthengine-legacy/assets/projects/soils-revealed/experimental-dataset', 'projects/earthengine-legacy/assets/projects/soils-revealed/nlcd_change_2011_20']


###  Recursively Map Assets

The following script will traverse all subfolders, image collections, and images in the legacy project and store the structure.

In [33]:
def map_assets_recursive(asset_path, asset_map=None):
    if asset_map is None:
        asset_map = {}
    
    # List assets at the current path
    assets = ee.data.listAssets({'parent': asset_path})
    
    # Map out each asset
    for asset in assets.get('assets', []):
        asset_type = asset['type']
        asset_name = asset['name']
        
        if asset_type == 'FOLDER':
            # If it's a folder, recursively map its contents
            print(f"Found folder: {asset_name}")
            asset_map[asset_name] = map_assets_recursive(asset_name, {})
        elif asset_type == 'IMAGE_COLLECTION':
            # Map Image Collections
            print(f"Found image collection: {asset_name}")
            asset_map[asset_name] = list_image_collection(asset_name)
        elif asset_type == 'IMAGE':
            # Map Images
            print(f"Found image: {asset_name}")
            asset_map[asset_name] = 'IMAGE'
        else:
            print(f"Skipping unknown type: {asset_type} at {asset_name}")
    
    return asset_map

def list_image_collection(collection_path):
    # List images in an Image Collection
    images = ee.ImageCollection(collection_path).aggregate_array('system:id').getInfo()
    return images

# Map assets in the legacy project
legacy_project = 'projects/earthengine-legacy/assets/projects/soils-revealed'
asset_map = map_assets_recursive(legacy_project)

# Print the asset map
print(asset_map)

# Save to a file
import json
with open('asset_map.json', 'w') as f:
    json.dump(asset_map, f, indent=2)

Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/ESA_landcover_ipcc
Found folder: projects/earthengine-legacy/assets/projects/soils-revealed/Future
Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/Future/scenario_crop_I_dSOC
Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/Future/scenario_crop_MGI_dSOC
Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/Future/scenario_crop_MG_dSOC
Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/Future/scenario_degradation_ForestToCrop_dSOC
Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/Future/scenario_degradation_ForestToGrass_dSOC
Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/Future/scenario_degradation_NoDeforestation_dSOC
Found image collection: projects/earthengine-legacy/assets/projects/soils-revealed/Future/s

## Step 3: Replicate the Structure in the Cloud Project
The following script uses the asset map to recreate the structure in your cloud projec

In [49]:
def create_asset_structure(asset_map, source_project, destination_project):
    for asset, content in asset_map.items():
        # Determine source and destination paths
        source_path = asset
        destination_path = asset.replace(f'{source_project}/', f'{destination_project}/')
        
        if isinstance(content, dict):  # It's a folder
            print(f"Creating folder: {destination_path}")
            try:
                ee.data.createFolder(destination_path)
            except Exception as e:
                print(f"Error creating folder {destination_path}: {e}")
            # Recur into the folder
            create_asset_structure(content, source_project, destination_project)
        elif isinstance(content, list):  # It's an Image Collection
            print(f"Creating image collection: {destination_path}")
            try:
                ee.data.createAsset({'type': 'IMAGE_COLLECTION'}, destination_path)
            except Exception as e:
                print(f"Error creating collection {destination_path}: {e}")
            # Copy each image in the collection
            for image in content:
                dest_image_path = image.replace('projects/soils-revealed/', f'{destination_project}/')
                try:
                    print(f"Copying image: {image} to {dest_image_path}")
                    if image != dest_image_path:  # Avoid self-copying
                        ee.data.copyAsset(image, dest_image_path)
                    else:
                        print(f"Skipped copying {image} to itself.")
                except Exception as e:
                    print(f"Error copying image {image}: {e}")
        elif content == 'IMAGE':  # It's an individual image
            destination_path = source_path.replace('projects/soils-revealed/', f'{destination_project}/')
            print(f"Copying image: {source_path} to {destination_path}")
            try:
                if source_path != destination_path:  # Avoid self-copying
                    ee.data.copyAsset(source_path, destination_path)
                else:
                    print(f"Skipped copying {source_path} to itself.")
            except Exception as e:
                print(f"Error copying image {source_path}: {e}")
                
# Define source and destination projects
source_project = 'projects/earthengine-legacy/assets/projects/soils-revealed'
destination_project = 'projects/soils-revealed-307010/assets'

# Replicate the structure
create_asset_structure(asset_map, source_project, destination_project)

Creating image collection: projects/soils-revealed-307010/assets/ESA_landcover_ipcc
Error creating collection projects/soils-revealed-307010/assets/ESA_landcover_ipcc: Cannot overwrite asset 'projects/soils-revealed-307010/assets/ESA_landcover_ipcc'.
Copying image: projects/soils-revealed/ESA_landcover_ipcc/ESA_2000_ipcc to projects/soils-revealed-307010/assets/ESA_landcover_ipcc/ESA_2000_ipcc
Copying image: projects/soils-revealed/ESA_landcover_ipcc/ESA_2001_ipcc to projects/soils-revealed-307010/assets/ESA_landcover_ipcc/ESA_2001_ipcc
Copying image: projects/soils-revealed/ESA_landcover_ipcc/ESA_2002_ipcc to projects/soils-revealed-307010/assets/ESA_landcover_ipcc/ESA_2002_ipcc
Copying image: projects/soils-revealed/ESA_landcover_ipcc/ESA_2003_ipcc to projects/soils-revealed-307010/assets/ESA_landcover_ipcc/ESA_2003_ipcc
Copying image: projects/soils-revealed/ESA_landcover_ipcc/ESA_2004_ipcc to projects/soils-revealed-307010/assets/ESA_landcover_ipcc/ESA_2004_ipcc
Copying image: proj